
### <center>Procesamiento Digital de Señales de Audio</center>
#### <center>Instituto de Ingeniería Eléctrica - UdelaR</center>
# Hoja de Ejercicios 4 - Curso 2024
### Análisis homomórfico
### Análisis por predicción lineal


## Pautas para el práctico
 - La realización del presente trabajo es individual.
 - Es necesario abordar todos los ejercicios propuestos.
 - Se espera la entrega de un PDF escrito en $\LaTeX$ o similar. El mismo tendrá:
     - Máximo de 14 páginas
     - Máximo de 2500 palabras
 - También se espera la entrega del código escrito, en scripts Python o en este mismo Jupyter Notebook.
 - La corrección del práctico se hará sobre lo entregado en el PDF, pero podremos apoyarnos en el razonamiento y comprensión demostrado en el código escrito. Recomendamos escribir el código de forma prolija para facilitar la comprensión presente y futura tanto de nosotros como de ustedes.
 - Los ejercicios marcados como $\blacklozenge$ son opcionales.


**Nombre de el/la estudiante:** 

### Como correr este notebook

Es posible descargarlo y correrlo localmente en su computadora

Tambien pueden correrlo en Google Colab usando el siguiente link.

<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/mrocamora/audio-dsp/blob/main/practicos/AudioDSP_Practico_4.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Correr en Google Colab</a>
  </td>
</table>

In [1]:
# # Al correr esta celda, se podrá acceder a archivos
# # y carpetas en su cuenta de google drive.
# # Puede ver la estructura de carpetas apretando en
# # el icono de carpeta de la barra lateral izquierda.

# try:
#     from google.colab import drive
#     drive.mount('/content/drive')

#     ### Camino a carpeta de los archivos (ajustarlo segun corresponda)
#     dir_files = './drive/MyDrive/DSPAudio/Practico4/Archivos_P4/'

# except:
#   dir_files = './Archivos_P4/'

# print(f'Files path: {dir_files}')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import io, signal
from scipy.linalg import solve_toeplitz
from scipy.io.wavfile import read
from IPython.display import Audio

# Ejercicio 1

En este ejercicio se estudia el cepstrum de señales de audio. Según el modelo del mecanismo de producción de la voz, la señal de voz se puede expresar como $s[n]=p[n]*h[n]$, donde $p[n]$ es la señal de excitación y $h[n]$ es la respuesta al impulso del tracto vocal. Mediante el cepstrum complejo se pretende deconvolucionar la señal de voz en la excitación y la respuesta al impulso.


## Parte 1


1. En el caso de sonidos sonoros, la excitación $p[n]$ es un tren de pulsos periódico,

    $$p[n]=\beta^n\sum_{k=0}^{\infty}\delta[n-kP]$$

    Calcular analíticamente el cepstrum complejo $\hat{p}[n]$ de $p[n]$. Graficar empleando los valores $\beta = 0.99$ y $P=100$.

1. Calcular analíticamente el cepstrum complejo $\hat{h}[n]$ de la secuencia $h[n]$ cuya transformada $\mathcal{Z}$ es
  $$ H(z) = \frac{(1-bz)(1-b^*z)}{(1-cz^{-1})(1-c^*z^{-1})},\;\;\;\textrm{con }|b|,|c|<1 $$

$\qquad$ Graficar empleando los valores $b=0.98e^{j0.81\pi}$ y $c=0.98e^{j0.19\pi}$.

1. Considere ahora la señal $s[n]=h[n]*p[n]$. Calcular analíticamente el cepstrum $\hat{s}[n]$ de $s[n]$.

1. Calcular el cepstrum complejo de las señales $p[n]$ y $h[n]$ utilizando la Transformada Discreta Fourier. Comparar con el resultado analítico y comentar los resultados.

1. Se desea recuperar la respuesta al impulso $h[n]$ a partir de la señal $s[n]$. Para hacerlo, liftrar el cepstrum complejo $\hat{s}[n]$ apropiadamente eliminando los componentes de altas quefrencys y aplicar el cepstrum inverso. Comparar gráficamente la respuesta al impulso recuperada con la respuesta al impulso verdadera.


## Parte 2
Algunas aplicaciones del cepstrum real en señales de voz son la estimación de la frecuencia fundamental y la detección de formantes. Para eso, se procesa la señal en fragmentos de tiempo corto y se calcula el cepstrum real de cada fragmento. La presencia de un pico en la región de medianas o altas quefrencys es un indicador de sonoridad y la quefrency del pico indica el período.

Se sugiere seguir los siguientes pasos para estimar la evolución de la frecuencia fundamental de la señal de voz del archivo [*tamy-vocals.wav*](./Archivos_P4/problema1/tamy-vocals.wav)

  1. Calcular el cepstrum de tiempo corto de la señal. Graficar el resultado en el plano tiempo-quefrency eliminando los componentes de bajas quefrencys para la correcta visualización del cepstrum del tren de pulsos periódico en las regiones sonoras.
  1. A partir de la presencia y posición del pico construir un algoritmo para la detección de sonoridad y frecuencia fundamental. Establecer la frecuencia fundamental en 0 Hz en las regiones en donde el sonido es sordo. Comparar el resultado con el [*ground-truth*](./Archivos_P4/problema1/tamy-vocals_ground_truth.csv).
  



# Ejercicio 2

## Parte 1

En el modelo de predicción lineal se asume que la muestra actual de la señal de voz $s[n]$ es predecible a partir de una combinación lineal de $p$ muestras previas,

$$\tilde{s}[n] = \sum_{k=1}^{p}\alpha_k s[n-k]$$

El problema consiste en encontrar los coeficientes $\alpha_k$ del predictor que mejor aproximan a la señal $s[n]$. Para eso se define el error de predicción como

$$e_n[m] = s_n[m] - \tilde{s}_n[m]$$

donde $s_n[m]$ es un fragmento de tiempo corto de la señal de voz elegido en torno a la muestra $n$.

Se define el error cuadrático medio de predicción como

$$E_n = \sum_m e_n^2[m]$$

para algún intervalo de muestras $m$ que no es necesario especificar por el momento. 
En el modelo de predicción lineal, el conjunto de coeficientes $\lbrace\hat{\alpha}_k\rbrace$ óptimo es el que minimiza el error cuadrático medio de predicción. Se pide: 


1. Demostrar que los coeficientes que minimizan el error cuadrático medio obedecen el siguiente sistema lineal de ecuaciones (*ecuaciones normales*),

    $$\sum_{k=1}^{p}\hat{\alpha}_k\sum_m s_n[m-i]s_n[m-k]=\sum_m s_n[m-i]s_n[m],\,\,\,1\leq i \leq p$$

1. Demostrar que el error cuadrático medio mínimo de predicción es

    $$E_n = \sum_m s_n^2[m]-\sum_{k=1}^p\hat{\alpha}_k\sum_m s_n[m]s_n[m-k]$$



## Parte 2

En este problema se aplica la técnica de LPC para la clasificación de vocales, usando una base de datos de vocales aisladas pronunciadas por dos hablantes.

El procedimiento consiste en calcular el modelo todo polos de la señal de voz, y a partir de los polos obtener la frecuencia de las dos primeras formantes $\left(F_1,\,F_2\right)$. 


A modo de referencia, en el cuadro de abajo se indica la frecuencia promedio de las dos primeras formantes de las vocales del idioma español (Estos datos son aproximaciones de los datos provistos en https://joaquimllisterri.cat/phonetics/fon_anal_acus/caract_acust.html). 


La señal analizada puede clasificarse a partir la vocal de referencia mas cercana en el plano $\left(F_1,\,F_2\right)$.

| Fonema | $F_1(Hz)$ | $F_2(Hz)$ |
|:------:|:---------:|:---------:|
|   /a/  |    800    |    1170   |
|   /e/  |    480    |    2300   |
|   /i/  |    240    |    2800   |
|   /o/  |    510    |    960    |
|   /u/  |    250    |    630    |

<center>**Primeras dos formantes de las vocales en el idioma español.**</center>


1. Implementar un algoritmo para procesar todas las señales de la base de datos, calculando para cada una la frecuencia de las dos primeras formantes. Mostrar los resultados como un mapa de formantes en el plano $\left(F_1,\,F_2\right)$.
1. Clasificar las señales a partir de las vocales de referencia. Reportar la tasa de acierto obtenida para cada vocal y para cada hablante.
1. Analizar los resultados y proponer alguna estrategia para mejorarlos. 
    

Las señales de la base de datos están muestreadas a 8000 Hz. Hay un directorio por hablante y el nombre de los archivo de audio es **[vocal]-[número].wav**, con **número** de 1 a 10. 
Los archivos contienen un único fonema, de duración variable (sin silencio al comienzo o al final), pero todos superan las 550 muestras.


Tener en cuenta los siguientes aspectos. 

- Se sugiere tomar una ventana centrada en la muestra central de cada señal. 
- Elegir adecuadamente el tamaño de la ventana $N$ y el orden $p$ del modelo.
- Calcular los polos y representarlos en un diagrama de polos y ceros. 
- Eliminar los polos reales y los polos con $\omega\geq\pi$ (Son redundantes por ser complejos conjugados de los polos con $\omega<\pi$). Eliminar los polos de ancho de banda mayor a cierto umbral. Elegir adecuadamente el umbral.
- Establecer la frecuencia de las dos primeras formantes como la frecuencia de los dos polos de menor frecuencia.


In [ ]:
### Cargar los audios y asignarle una etiqueta del tipo #hablante-vocal
vocales = ['a','e','i','o','u']
folders = ['/problema2/vocales/martin/','/problema2/vocales/cecilia/']
indices = np.arange(1,11)

files_list = []
labels = []

for folder in folders:
  for vocal in vocales:
    for indice in indices:
      _, audio = io.wavfile.read(dir_files + folder + vocal + '-' + str(indice) + '.wav')
      files_list.append(audio)
      labels.append(str(folders.index(folder)) + '-' + vocal)
      
# Frecuencia de muestreo
fs = 8000

In [ ]:
# Escuchar vocal del hablante 1
Audio(files_list[45], rate=fs)

In [ ]:
# Escuchar vocal del hablante 2
Audio(files_list[95], rate=fs)